# day_01.py 核心组件使用示例

本 Jupyter Notebook 提供了三个核心组件的详细使用示例：
1. **DistributedCacheDecorator** - 分布式缓存装饰器
2. **AsyncRAGGenerator** - 异步RAG流式生成器
3. **ConcurrentRouter** - 并发模型路由器

每个示例都包含详细注释和使用场景说明，可直接运行查看效果。

In [1]:
import asyncio
import time
from day_01 import DistributedCacheDecorator, AsyncRAGGenerator, ConcurrentRouter, ModelEndpoint

## 1. DistributedCacheDecorator 使用示例

**场景说明：**
- 场景1: 缓存大模型API调用结果
- 场景2: 缓存数据库查询结果
- 场景3: 缓存计算密集型函数结果

In [2]:
print("=== DistributedCacheDecorator 使用示例 ===")

# 场景1: 缓存大模型API调用结果
print("1. 缓存大模型API调用结果:")

@DistributedCacheDecorator(ttl=60, cache_backend='redis')
def fetch_llm_response(prompt: str, model: str = "gpt-4") -> dict:
    """模拟大模型API调用"""
    print(f"调用模型 {model}: {prompt[:50]}...")
    time.sleep(0.5)  # 模拟网络延迟
    return {"response": f"针对'{prompt}'的AI回复", "tokens": 150}

# 第一次调用，会执行函数
start_time = time.time()
result1 = fetch_llm_response("什么是Python?")
print(f"首次调用耗时: {time.time() - start_time:.2f}秒")
print(f"结果: {result1}")

# 第二次调用相同参数，会命中缓存
start_time = time.time()
result2 = fetch_llm_response("什么是Python?")
print(f"缓存命中耗时: {time.time() - start_time:.2f}秒")
print(f"结果: {result2}")

=== DistributedCacheDecorator 使用示例 ===
1. 缓存大模型API调用结果:
调用模型 gpt-4: 什么是Python?...
同步到分布式缓存: fetch_llm_response:-822733053630420067
首次调用耗时: 0.51秒
结果: {'response': "针对'什么是Python?'的AI回复", 'tokens': 150}
缓存命中: fetch_llm_response:-822733053630420067
缓存命中耗时: 0.00秒
结果: {'response': "针对'什么是Python?'的AI回复", 'tokens': 150}


In [ ]:
# 场景2: 缓存数据库查询结果
print("2. 缓存数据库查询结果:")

@DistributedCacheDecorator(ttl=300)  # 5分钟缓存
def query_database(user_id: int) -> dict:
    """模拟数据库查询"""
    print(f"查询用户 {user_id} 的数据...")
    time.sleep(0.3)  # 模拟数据库查询延迟
    return {"user_id": user_id, "name": f"用户{user_id}", "email": f"user{user_id}@example.com"}

# 第一次查询
start_time = time.time()
result = query_database(1001)
print(f"查询结果: {result}")
print(f"查询耗时: {time.time() - start_time:.3f}秒")

# 再次查询相同用户，会命中缓存
start_time = time.time()
result = query_database(1001)
print(f"缓存结果: {result}")
print(f"缓存命中耗时: {time.time() - start_time:.3f}秒")


2. 缓存数据库查询结果:
查询用户 1001 的数据...
同步到分布式缓存: query_database:-7274342873189353207
查询结果: {'user_id': 1001, 'name': '用户1001', 'email': 'user1001@example.com'}
缓存命中: query_database:-7274342873189353207
缓存结果: {'user_id': 1001, 'name': '用户1001', 'email': 'user1001@example.com'}


In [ ]:
# 场景3: 缓存计算密集型函数结果
print("3. 缓存计算密集型函数结果:")

@DistributedCacheDecorator(ttl=600)  # 10分钟缓存
def compute_expensive(x: int, y: int) -> int:
    """模拟计算密集型操作"""
    print(f"执行密集计算: {x} + {y}...")
    time.sleep(1.0)  # 模拟计算延迟
    return x + y

# 第一次计算
start_time = time.time()
result = compute_expensive(1000000, 2000000)
print(f"计算结果: {result}")
print(f"计算耗时: {time.time() - start_time:.2f}秒")

# 再次计算相同参数，会命中缓存
start_time = time.time()
result = compute_expensive(1000000, 2000000)
print(f"缓存结果: {result}")
print(f"缓存耗时: {time.time() - start_time:.2f}秒")

## 2. AsyncRAGGenerator 使用示例

**场景说明：**
- 场景1: 基本流式检索
- 场景2: 处理多个文档
- 场景3: 结合实际应用场景

In [ ]:
async def example_async_rag_generator():
    print("\n=== AsyncRAGGenerator 使用示例 ===")
    
    # 创建RAG生成器实例
    generator = AsyncRAGGenerator(chunk_size=200)
    
    # 场景1: 基本流式检索
    print("\n1. 基本流式检索:")
    
    # 准备测试文档
    documents = [
        "Python是一种广泛使用的高级编程语言，由Guido van Rossum创建。Python的设计哲学强调代码的可读性和简洁的语法。",
        "Python支持多种编程范式，包括面向对象、命令式、函数式和过程式编程。Python是一种解释型语言，意味着代码可以直接运行而不需要编译。",
        "Python拥有丰富的标准库和第三方库，使其成为数据科学、人工智能、Web开发等领域的热门选择。"
    ]
    
    # 流式检索
    print("查询: Python 编程语言")
    async for result in generator.stream_retrieval("Python 编程语言", documents):
        print(f"  {result}")
    
    # 场景2: 处理多个文档
    print("\n2. 处理多个文档:")
    
    # 准备更多文档
    more_documents = [
        "机器学习是人工智能的一个分支，专注于开发能够从数据中学习的算法。",
        "深度学习是机器学习的一个子集，使用多层神经网络来模拟人脑的学习过程。",
        "Python是机器学习和深度学习的首选语言，拥有丰富的库如TensorFlow、PyTorch和scikit-learn。",
        "数据科学是一个跨学科领域，结合了统计学、计算机科学和领域知识来提取数据中的 insights。"
    ]
    
    # 流式检索
    print("查询: Python 机器学习")
    async for result in generator.stream_retrieval("Python 机器学习", more_documents):
        print(f"  {result}")
    
    # 场景3: 结合实际应用场景
    print("\n3. 结合实际应用场景:")
    
    async def rag_assistant(query: str, documents: list) -> str:
        """RAG助手：使用文档信息回答问题"""
        print(f"\n用户问题: {query}")
        print("正在检索相关信息...")
        
        # 收集相关信息
        relevant_info = []
        async for chunk in generator.stream_retrieval(query, documents):
            relevant_info.append(chunk)
        
        # 生成回答
        if relevant_info:
            print("找到相关信息，生成回答...")
            return f"根据检索到的信息，我可以回答您的问题: {query}\
\
相关信息:\
" + "\n".join(relevant_info)
        else:
            return f"抱歉，我没有找到与 '{query}' 相关的信息。"
    
    # 测试RAG助手
    all_documents = documents + more_documents
    response = await rag_assistant("Python在机器学习中的应用", all_documents)
    print(f"\n助手回答: {response}")

# 运行异步RAG生成器示例
await example_async_rag_generator()

## 3. ConcurrentRouter 使用示例

**场景说明：**
- 场景1: 基本路由
- 场景2: 首选模型设置
- 场景3: 自动降级
- 场景4: 并发控制

In [ ]:
async def example_concurrent_router():
    print("\n=== ConcurrentRouter 使用示例 ===")
    
    # 创建模型端点
    endpoints = [
        ModelEndpoint(name="gpt-4", url="https://api.openai.com/v1/chat/completions", max_concurrent=5),
        ModelEndpoint(name="claude-3", url="https://api.anthropic.com/v1/messages", max_concurrent=3),
        ModelEndpoint(name="gemini", url="https://generativeai.googleapis.com/v1/models/gemini-pro:generateContent", max_concurrent=4)
    ]
    
    # 创建路由器
    router = ConcurrentRouter(endpoints)
    
    # 场景1: 基本路由
    print("\n1. 基本路由:")
    result = await router.route_request("你好，请介绍一下自己")
    print(f"使用模型: {result['model']}")
    print(f"回复: {result['response']}")
    
    # 场景2: 首选模型设置
    print("\n2. 首选模型设置:")
    result = await router.route_request("什么是量子计算?", preferred_model="gpt-4")
    print(f"使用模型: {result['model']}")
    print(f"回复: {result['response']}")
    
    # 场景3: 自动降级（模拟模型失败）
    print("\n3. 自动降级:")
    
    # 保存原始调用方法
    original_call_model = router._call_model
    
    async def mock_call_model(model_name, prompt):
        """模拟模型调用，使gpt-4失败"""
        if model_name == "gpt-4":
            print(f"模拟模型 {model_name} 失败")
            raise Exception("Model service unavailable")
        return await original_call_model(model_name, prompt)
    
    # 替换调用方法
    router._call_model = mock_call_model
    
    # 尝试使用首选模型gpt-4，应该会自动降级到其他模型
    try:
        result = await router.route_request("什么是人工智能?", preferred_model="gpt-4")
        print(f"使用模型: {result['model']}")
        print(f"回复: {result['response']}")
        print("✓ 自动降级成功")
    except Exception as e:
        print(f"✗ 自动降级失败: {e}")
    
    # 恢复原始调用方法
    router._call_model = original_call_model
    
    # 场景4: 并发控制
    print("\n4. 并发控制:")
    
    # 创建一个最大并发为1的模型端点
    limited_endpoints = [
        ModelEndpoint(name="limited-model", url="http://example.com/model", max_concurrent=1)
    ]
    limited_router = ConcurrentRouter(limited_endpoints)
    
    # 定义并发请求函数
    async def make_request(i):
        print(f"发起请求 {i}...")
        start = time.time()
        result = await limited_router.route_request(f"测试请求 {i}")
        end = time.time()
        print(f"请求 {i} 完成，使用模型: {result['model']}，耗时: {end - start:.2f}秒")
        return result
    
    # 并发执行3个请求
    print("执行3个并发请求（最大并发限制为1）:")
    start_time = time.time()
    results = await asyncio.gather(
        make_request(1),
        make_request(2),
        make_request(3)
    )
    total_time = time.time() - start_time
    print(f"\n所有请求完成，总耗时: {total_time:.2f}秒")
    print("注: 由于并发限制为1，请求会串行执行")

# 运行并发模型路由器示例
await example_concurrent_router()

## 总结

本 Notebook 演示了三个核心组件的使用方法：

1. **DistributedCacheDecorator**：
   - 缓存大模型API调用结果，提升性能
   - 缓存数据库查询结果，减少重复查询
   - 缓存计算密集型函数结果，优化计算效率

2. **AsyncRAGGenerator**：
   - 流式检索相关文档片段
   - 处理多个文档的检索
   - 结合实际应用场景，构建RAG问答助手

3. **ConcurrentRouter**：
   - 智能路由模型请求
   - 设置首选模型
   - 实现模型失败时的自动降级
   - 控制并发请求数量

这些组件可以根据实际需求进行定制和扩展，为构建企业级AI Agent系统提供了坚实的基础。